<a id=top></a>
# Pea3 smFISH Analysis

## Table of Contents

----

1. [Preparations](#prep)
2. [QC: Spot Detection](#QC_spots)
3. [QC: Cell Shape](#QC_shape)
4. [Data Visualization](#viz)
5. [Predicting Expression from Shape: Testing](#atlas_test)
6. [Predicting Expression from Shape: Running](#atlas_run)
7. [Predicting Expression from Shape: Visualization](#atlas_viz)

<a id=prep></a>

## 1. Preparations
[back to top](#top)

----

In [ ]:
### Import modules

# External, general
from __future__ import division
import os, sys, pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# External, specific
import ipywidgets as widgets
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skimage import io
from sklearn import model_selection, metrics, multioutput
import sklearn.svm as svm

# Internal
import katachi.utilities.loading as ld

In [ ]:
### Load general data

# Prep loader
loader = ld.DataLoaderIDR()
loader.find_imports(r"data/experimentB/extracted_measurements/", recurse=True, verbose=True)

# Import shape spaces
fspace_TFOR, prim_IDs, fspace_idx = loader.load_dataset("shape_TFOR_raw_measured.tsv")
fspace_CFOR, _, _ = loader.load_dataset("shape_CFOR_raw_measured.tsv", IDs=prim_IDs)
print "Imported TFOR shape space of shape:", fspace_TFOR.shape
print "Imported CFOR shape space of shape:", fspace_CFOR.shape

# Standardization and PCA
fspace_TFOR_z = StandardScaler().fit_transform(fspace_TFOR)
pca_TFOR = PCA()
fspace_TFOR_pca = pca_TFOR.fit_transform(fspace_TFOR_z)
fspace_CFOR_z = StandardScaler().fit_transform(fspace_CFOR)
pca_CFOR = PCA()
fspace_CFOR_pca = pca_CFOR.fit_transform(fspace_CFOR_z)

# Import TFOR centroid locations
centroids = loader.load_dataset("_other_measurements.tsv", IDs=prim_IDs)[0][:,3:6][:,::-1]
print "Imported TFOR centroids of shape:", centroids.shape

# Import & standardize engineered features
covar_df, _, _ = loader.load_dataset("_other_measurements.tsv", IDs=prim_IDs, force_df=True)
del covar_df['Centroids RAW X']; del covar_df['Centroids RAW Y']; del covar_df['Centroids RAW Z']
covar_names = list(covar_df.columns)
covar_df_z = (covar_df - covar_df.mean()) / covar_df.std()
print "Imported engineered features of shape:", covar_df.shape

In [ ]:
### Load smFISH data

# Counts
rna_counts, _, _ = loader.load_dataset("pea3smFISH_RNAcounts_measured.tsv", IDs=prim_IDs)
print "Imported RNA counts data of shape:", rna_counts.shape

# Spots
rna_spots, _, _= loader.load_dataset("pea3smFISH_RNAspot_coordinates.tsv", IDs=prim_IDs, force_dict=True)
print "Imported RNA spot coordinates for", len(rna_spots), "samples, the first having shape", rna_spots[prim_IDs[0]].shape

In [ ]:
### Outlier removal

# Remove samples with `mean(rna_counts) <= mean_count_thresh` as a simple and helpful quality threshold
mean_count_thresh = 2
count_means = [np.mean(rna_counts[fspace_idx==prim_idx]) for prim_idx in range(len(prim_IDs))]
rna_exclude_prim_mask = np.array(count_means) > mean_count_thresh
rna_exclude_cell_mask = rna_exclude_prim_mask[fspace_idx]

# Report
print "Excluding", np.sum(~rna_exclude_prim_mask), "prims /", np.sum(~rna_exclude_cell_mask), "cells,",
print "resulting in", np.sum(rna_exclude_prim_mask), "prims /", np.sum(rna_exclude_cell_mask), "cells",
print "left for analysis."

<a id=QC_spots></a>

## 2. QC: Spot Detection
[back to top](#top)

----

In [ ]:
### Boxplot of mean counts & per-cell counts

# Note: 
#  - Durdu et al. found a mean of ~11 spots/cell in their manually analyzed data.
#    This plot is designed to fit their way of reporting the results.
#  - This is recapitulated quite well here, except for a couple of outliers with 
#    unrealistically low expression.
#  - However, note that the cell-level distribution is very non-normal, so the mean 
#    is not a very good summary characteristic.

# Get count means
count_means = np.array([np.mean(rna_counts[fspace_idx==prim_idx]) 
                        for prim_idx in range(len(prim_IDs))])

# Fig prep
fig, ax = plt.subplots(1, 2, figsize=(3.5, 4.5))

# Make boxplots
bp_m = ax[0].boxplot(count_means, widths=0.5, patch_artist=True)
bp_a = ax[1].boxplot(rna_counts, widths=0.5, patch_artist=True, showfliers=False)

# Boxplot styling function (making it similar to Sevi's paper)
def style_boxplot(bp):
    for patch in bp['boxes']:
        patch.set(edgecolor='black', linewidth=1.2,)
    for whisker in bp['whiskers']:
        whisker.set(color='black', linestyle='-')
    for cap in bp['caps']:
        cap.set(linewidth=1.2)
    for median in bp['medians']:
        median.set(color='black', linewidth=1.2)
        
# Style the boxplots
style_boxplot(bp_m)
style_boxplot(bp_a)

# Add scatter
ax[0].scatter(np.random.normal(1.0, 0.06, len(count_means)), count_means, 
              zorder=10, s=20, alpha=0.7, c='midnightblue', edgecolor='')
ax[0].set_ylim([-2, 47])
ax[1].scatter(np.random.normal(1.0, 0.06, len(rna_counts)), rna_counts, 
              zorder=10, s=2, alpha=0.1, c='midnightblue', edgecolor='')
ax[1].set_ylim([-2, 100])

# Remove ticks
ax[0].yaxis.set_ticks_position('left')
ax[0].xaxis.set_ticks_position('bottom')
ax[1].yaxis.set_ticks_position('left')
ax[1].xaxis.set_ticks_position('bottom')

# Axis labels
from matplotlib import rcParams
rcParams['mathtext.default'] = 'regular'
ax[0].set_ylabel(r'$\it{pea3}$ transcripts per cell (mean)', fontsize=12, labelpad=5)
ax[0].set_xticklabels(['WT 880'], rotation=90, fontsize=12)
ax[1].set_ylabel(r'$\it{pea3}$ transcripts per cell (all)', fontsize=12, labelpad=0)
ax[1].set_xticklabels(['WT 880'], rotation=90, fontsize=12)
plt.tight_layout()

# Show
plt.show()

In [ ]:
### Histograms of RNA counts for each sample

# Prep
n_plot_cols = 7
n_plot_rows = int(np.ceil(len(prim_IDs) / n_plot_cols))
fig, ax = plt.subplots(n_plot_rows, n_plot_cols, figsize=(1.5*n_plot_cols, 1.5*n_plot_rows),
                       sharex=True, sharey=True)
ax = ax.flatten()
[ax[i].axis('off') for i in range(len(prim_IDs), n_plot_cols*n_plot_rows)]

# For each sample...
for axx, prim_idx, prim_ID, is_outlier in zip(ax, range(len(prim_IDs)), prim_IDs, ~rna_exclude_prim_mask):
    
    # Generate the histogram
    axx.hist(rna_counts[fspace_idx==prim_idx], 
             bins=40, range=(rna_counts.min(), rna_counts.max()),
             histtype='stepfilled', color='darkblue' if not is_outlier else 'darkred', alpha=0.5)
    axx.set_title(prim_ID, fontsize=9)

# Set common axis labels
fig.text(0.5, -0.01, 'RNA Counts', ha='center', va='center')
fig.text(-0.01, 0.50, 'Histogram\nof Cells',  ha='center', va='center', rotation='vertical')

# Done
plt.tight_layout()
plt.show()

In [ ]:
### Histogram of counts over all cells

# Prep
plt.figure(figsize=(5, 3))

# Make hist
plt.hist(rna_counts, bins=100, histtype='stepfilled', color='b', alpha=0.5)

# Label
plt.xlabel('RNA Count')
plt.ylabel('Histogram of Cells')

# Done
plt.show()

<a id=QC_shape></a>

## 3. QC: Cell Shape (Fixation Effects)
[back to top](#top)


----

In [ ]:
### Load live imaging reference data

# Prep loader
ref_loader = ld.DataLoaderIDR()
ref_loader.find_imports(r"data/experimentA/extracted_measurements/", recurse=True, verbose=True)

# Use only the 24 samples that were single-color imaged
ref_IDs = ['056F63395C', '08B96BE794', '0B51F8B46C', '1C43D83E9A', '2902E38204', '4DC24FC301',  
           '6F18162F4C', '8C633380D2', 'B95A4F6D95', 'CB87D7CBC9', '0E48AB134C', '3612A6CEF5', 
           '8713481504', '8C83D4387F', 'AB98466077', 'C95F528559', 'E013272A99', 'E6E56C3F42', 
           '22DF2AE1A0', '2B23352582', '673A65D087', '8CA33561B5', 'EC77708A51', 'FC90367714']

# Import shape spaces
ref_TFOR, _, ref_idx = ref_loader.load_dataset("shape_TFOR_raw_measured.tsv", IDs=ref_IDs)
ref_CFOR, _, _ = ref_loader.load_dataset("shape_CFOR_raw_measured.tsv", IDs=ref_IDs)
print "Imported TFOR shape space of shape:", ref_TFOR.shape
print "Imported CFOR shape space of shape:", ref_CFOR.shape

# Standardization and apply PCA (fitted above)
ref_TFOR_z = StandardScaler().fit_transform(ref_TFOR)
ref_TFOR_pca = pca_TFOR.transform(ref_TFOR_z)
ref_CFOR_z = StandardScaler().fit_transform(ref_CFOR)
ref_CFOR_pca = pca_CFOR.transform(ref_CFOR_z)

# Import & standardize engineered features
ref_covar_df, _, _ = ref_loader.load_dataset("_other_measurements.tsv", IDs=ref_IDs, force_df=True)
del ref_covar_df['Centroids RAW X']; del ref_covar_df['Centroids RAW Y']; del ref_covar_df['Centroids RAW Z']
ref_covar_names = list(ref_covar_df.columns)
ref_covar_df_z = (ref_covar_df - ref_covar_df.mean()) / ref_covar_df.std()
print "Imported engineered features of shape:", ref_covar_df.shape

In [ ]:
### Compare to reference shape spaces: overlay

# Set interactions
@widgets.interact(PCx=(1, fspace_TFOR_pca.shape[1], 1),
                  PCy=(1, fspace_TFOR_pca.shape[1], 1))

# Show 
def show_PCs(PCx=1, PCy=2): 
    
    # Prep
    fig, ax = plt.subplots(1, 2, figsize=(12,5))
        
    # Plot TFOR
    ax[0].scatter(ref_TFOR_pca[:,PCx-1], ref_TFOR_pca[:,PCy-1],
                c='b', cmap=plt.cm.plasma, edgecolor='', 
                s=20, alpha=0.25, label='reference')
    ax[0].scatter(fspace_TFOR_pca[:,PCx-1], fspace_TFOR_pca[:,PCy-1],
                c='r', cmap=plt.cm.plasma, edgecolor='',
                s=20, alpha=0.25, label='fixed')
    
    # Plot CFOR
    ax[1].scatter(ref_CFOR_pca[:,PCx-1], ref_CFOR_pca[:,PCy-1],
                c='b', cmap=plt.cm.plasma, edgecolor='', 
                s=20, alpha=0.25, label='reference')
    ax[1].scatter(fspace_CFOR_pca[:,PCx-1], fspace_CFOR_pca[:,PCy-1],
                c='r', cmap=plt.cm.plasma, edgecolor='',
                s=20, alpha=0.25, label='fixed')

    # Cosmetics
    ax[0].legend(fontsize=8, frameon=False)
    ax[0].set_xlabel("PC "+str(PCx))
    ax[1].set_xlabel("PC "+str(PCx))
    ax[0].set_ylabel("PC "+str(PCy))
    ax[1].set_ylabel("PC "+str(PCy))
    ax[0].set_title("TFOR")
    ax[1].set_title("CFOR")
    
    # Done
    plt.tight_layout()
    plt.show()

In [ ]:
### Compare to reference cell extents

# Prep for plots
fig, ax = plt.subplots(1, 3, figsize=(6.5,3), sharey=True)

# Create plots
for i, lbl in enumerate(['Z', 'Y', 'X']):

    # Violinplot
    vio = ax[i].violinplot([ref_covar_df[lbl+' Axis Length'], 
                            covar_df[lbl+' Axis Length']], 
                           widths=0.60, showextrema=False)

    # Violinplot cosmetics
    vio['bodies'][0].set_facecolors('lightskyblue')
    vio['bodies'][1].set_facecolors('tomato')
    ax[i].set_xlim(0.3, 2.7)
    ax[i].set_xticks([1.0, 2.0])
    ax[i].set_xticklabels(["Reference", "Fixed"])
    ax[i].set_ylabel(lbl)

    # Jitter
    for j,y in enumerate([ref_covar_df[lbl+' Axis Length'], covar_df[lbl+' Axis Length']]):
        x = np.random.normal(j+1, 0.08, size=len(y))
        ax[i].plot(x, y, '.', color=['blue', 'red'][j], alpha=[0.1, 0.1][j], ms=2)

# Done
plt.tight_layout()
plt.show()

In [ ]:
### Compare to reference cell sphericity

# Violinplot
plt.figure(figsize=(2,3))
vio = plt.violinplot([ref_covar_df['Sphericity'], covar_df['Sphericity']], 
                     widths=0.60, showextrema=False)

# Violinplot cosmetics
vio['bodies'][0].set_facecolors('lightskyblue')
vio['bodies'][1].set_facecolors('tomato')
plt.xlim(0.3, 2.7)
plt.xticks([1.0, 2.0])
plt.gca().set_xticklabels(["Reference", "Fixed"])
plt.ylabel("Cell Sphericity")

# Jitter
for i,y in enumerate([ref_covar_df['Sphericity'], covar_df['Sphericity']]):
    x = np.random.normal(i+1, 0.08, size=len(y))
    plt.plot(x, y, '.', color=['blue', 'red'][i], alpha=[0.1, 0.1][i], ms=2)

# Done
plt.show()

In [ ]:
### Compare to reference cell volume

# Violinplot
plt.figure(figsize=(2,3))
vio = plt.violinplot([ref_covar_df['Volume'], covar_df['Volume']], widths=0.60, showextrema=False)

# Violinplot cosmetics
vio['bodies'][0].set_facecolors('lightskyblue')
vio['bodies'][1].set_facecolors('tomato')
plt.xlim(0.3, 2.7)
plt.xticks([1.0, 2.0])
plt.gca().set_xticklabels(["Reference", "Fixed"])
plt.ylabel("Cell Volume")

# Jitter
for i,y in enumerate([ref_covar_df['Volume'], covar_df['Volume']]):
    x = np.random.normal(i+1, 0.08, size=len(y))
    plt.plot(x, y, '.', color=['blue', 'red'][i], alpha=[0.1, 0.1][i], ms=2)

# Done
plt.show()

In [ ]:
### For publication: compare to diverse set of shape references

# Prep for plots
fig, ax = plt.subplots(1, 3, figsize=(8, 3.5))

# Violinplot
vio_data = [[ref_TFOR_pca[:,0], fspace_TFOR_pca[:,0]],  # TFOR PC 1
            [ref_CFOR_pca[:,0], fspace_CFOR_pca[:,0]],  # CFOR PC 1
            [ref_covar_df['Z Axis Length'], covar_df['Z Axis Length']]]  # Cell Height

# Create plots
for i, lbl in enumerate(['TFOR-PC1 (D-V orient.)', 
                         'CFOR-PC1 (sphericity)', 
                         r'Cell height $\it{[\mu m]}$']):

    # Violinplot
    vio = ax[i].violinplot(vio_data[i], widths=0.70, showextrema=False)

    # Violinplot cosmetics
    vio['bodies'][0].set_facecolors('w')
    vio['bodies'][1].set_facecolors('w')
    ax[i].set_xlim(0.3, 2.7)
    ylims = ax[i].get_ylim()
    ax[i].set_ylim(ylims[0]-(ylims[1]-ylims[0])*0.05, ylims[1]+(ylims[1]-ylims[0])*0.2)
    ax[i].set_xticks([1.0, 2.0])
    ax[i].set_xticklabels(["Live", "Fixed"], fontsize=14)
    ax[i].set_ylabel(lbl, fontsize=14, labelpad=0)
    ax[i].set_yticklabels([int(n) for n in ax[i].get_yticks()], fontsize=14)
    
    # Jitter
    for j,y in enumerate(vio_data[i]):
        x = np.random.normal(j+1, 0.08, size=len(y))
        ax[i].plot(x, y, '.', color=['blue', 'midnightblue'][j], alpha=[0.1, 0.1][j], ms=2)
        
    # Print stats
    print 'pMWU('+lbl+'):', stats.mannwhitneyu(*vio_data[i], alternative='two-sided')[1]

# Cosmetics
plt.tight_layout()

# Done
plt.show()

<a id=viz></a>

## 4. Data Visualization
[back to top](#top)

----

In [ ]:
### Overlay of counts on shape spaces

# Set interactions
@widgets.interact(PCx=(1, fspace_TFOR_pca.shape[1], 1),
                  PCy=(1, fspace_TFOR_pca.shape[1], 1),
                  vmax_factor=(0.0, 1.0, 0.1))

# Show 
def show_PCs(PCx=1, PCy=2, vmax_factor=0.5): 
    
    # Prep
    fig, ax = plt.subplots(1, 2, figsize=(12,5))
        
    # Plot TFOR
    ax[0].scatter(fspace_TFOR_pca[:,PCx-1], fspace_TFOR_pca[:,PCy-1],
                  c=rna_counts, cmap=plt.cm.plasma, 
                  vmax=vmax_factor*np.max(rna_counts),
                  s=20, edgecolor='', alpha=0.5)
    
    # Plot CFOR
    ax[1].scatter(fspace_CFOR_pca[:,PCx-1], fspace_CFOR_pca[:,PCy-1],
                  c=rna_counts, cmap=plt.cm.plasma,
                  vmax=vmax_factor*np.max(rna_counts),
                  s=20, edgecolor='', alpha=0.5)

    # Cosmetics 
    ax[0].set_xlabel("PC "+str(PCx))
    ax[1].set_xlabel("PC "+str(PCx))
    ax[0].set_ylabel("PC "+str(PCy))
    ax[1].set_ylabel("PC "+str(PCy))
    ax[0].set_title("TFOR")
    ax[1].set_title("CFOR")
    
    # Done
    plt.tight_layout()
    plt.show()

In [ ]:
### Tissue consensus map

# Note: This suffers a little because some prims are so weirdly angled in the images
#       that the TFOR transform didn't get them quite right.

# Settings
xlim = (-130,  8)
ylim = ( -19, 19)

# Exclude weirdly TFOR-ed prims (those with centroids of `x > 0`) for cleaner visualization
centroid_exclude_prim_mask = np.array([np.max(centroids[fspace_idx==prim_idx,-1]) 
                                      for prim_idx in range(len(prim_IDs))]) < 5
centroid_exclude_cell_mask = centroid_exclude_prim_mask[fspace_idx]
plot_exclude_cell_mask = rna_exclude_cell_mask & centroid_exclude_cell_mask

# Get plot values & remove outliers
plot_values = rna_counts[plot_exclude_cell_mask]

# Tools for smoothing on scatter
from katachi.utilities.pcl_helpers import pcl_gaussian_smooth
from scipy.spatial.distance import pdist, squareform

# Cut off at prim contour outline
kernel_prim = stats.gaussian_kde(centroids[plot_exclude_cell_mask,1:].T)
f_prim = kernel_prim(centroids[plot_exclude_cell_mask,1:].T)
f_prim_mask = f_prim > f_prim.min() + (f_prim.max()-f_prim.min())*0.1
plot_values    = plot_values[f_prim_mask]
plot_centroids = centroids[plot_exclude_cell_mask][f_prim_mask]

# Smoothen?
pdists = squareform(pdist(plot_centroids[:,1:]))
plot_values = pcl_gaussian_smooth(pdists, plot_values[:,np.newaxis], sg_percentile=0.5)[:,0]

# Initialize figure
fig, ax = plt.subplots(1, figsize=(8, 2.8))

# Contourf plot
cfset = ax.tricontourf(plot_centroids[:,2], plot_centroids[:,1], plot_values, 20, 
                       cmap='plasma', vmax=20) # Note: vmax manually set for consistency across plots!

# Illustrative centroids from a single prim
plt.scatter(centroids[fspace_idx==prim_IDs.index(prim_IDs[12]), 2], 
            centroids[fspace_idx==prim_IDs.index(prim_IDs[12]), 1],
            c='', alpha=0.5)

# Cosmetics
ax.set_xlabel('TFOR x', fontsize=16)
ax.set_ylabel('TFOR y', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=13)
plt.xlim(xlim); plt.ylim(ylim)
ax.invert_yaxis()  # To match images

# Colorbar
cbar = plt.colorbar(cfset, ax=ax, pad=0.01)
cbar.set_label('RNA Counts', rotation=270, labelpad=15, fontsize=16)
cbar.ax.tick_params(labelsize=13)

# Finalize
plt.tight_layout()

# Done
plt.show()

<a id=atlas_test></a>

## 5. Predicting Expression from Shape: Testing
[back to top](#top)

----

In [ ]:
### Settings, scoring & metrics

# General
use_PCs   = 10
num_CVs   = 5
test_size = 0.3

# Shuffle split for CV
cv_sets = model_selection.ShuffleSplit(n_splits=num_CVs, test_size=test_size, random_state=42)

# Prepare CV scorers
scoring = {'explained_variance'  : metrics.make_scorer(metrics.explained_variance_score),
           'mean_squared_error'  : metrics.make_scorer(metrics.mean_squared_error),
           'r2_score'            : metrics.make_scorer(metrics.r2_score)}

In [ ]:
### Various prep of feature/target spaces

# Prepare counts by adding 2nd dim
rna_counts_rdy = np.expand_dims(rna_counts, -1)

# Prepare location data by z-scoring
centroids_z = StandardScaler().fit_transform(centroids)

In [ ]:
### Remove prims/cells that were excluded as outliers

# Prepare fspaces & counts by removing excluded prims and subselecting PCs
rna_counts_rdy      = rna_counts_rdy[rna_exclude_cell_mask]
fspace_TFOR_pca_rdy = fspace_TFOR_pca[rna_exclude_cell_mask, :use_PCs]
fspace_CFOR_pca_rdy = fspace_CFOR_pca[rna_exclude_cell_mask, :use_PCs]
centroids_z_rdy     = centroids_z[rna_exclude_cell_mask]

In [ ]:
### Simple score reporting function

def report_score(scores, score_key):
    print "%s: %.3f +/- %.3f" % (score_key, np.mean(scores[score_key]), np.std(scores[score_key]))

#### Predicting expression from TFOR

In [ ]:
### Prepare single train-test split for visualization

# Split
out = model_selection.train_test_split(fspace_TFOR_pca_rdy, rna_counts_rdy, 
                                       test_size=test_size, random_state=42)
X_train, X_test, y_train, y_test = out

# Report
print "Final source fspace (full, train, test):", fspace_TFOR_pca_rdy.shape, X_train.shape, X_test.shape
print "Final target fspace (full, train, test):", rna_counts_rdy.shape, y_train.shape, y_test.shape

In [ ]:
# Hyperparameter screening for SVR

# Param grid
gd = 1.0 / X_test.shape[1]
param_grid = [{'C': [0.01, 0.1, 1.0, 10.0, 100.0], 
               'epsilon': [0.01, 0.1, 0.5, 1.0], 
               'gamma': [gd*10.0, gd, gd*0.1, gd*0.01]}]

# Prep regressor
svr = svm.SVR(kernel='rbf')

# Run grid search
clf = model_selection.GridSearchCV(svr, param_grid, 
                                   cv=cv_sets, scoring=scoring['explained_variance'],
                                   n_jobs=6, verbose=2)
clf.fit(fspace_TFOR_pca_rdy, rna_counts_rdy.ravel())

# Report
print "Best estimator:", clf.best_estimator_
print "Best score:", clf.best_score_

# Use best estimator for cross validation
svr = clf.best_estimator_
scores = model_selection.cross_validate(svr, fspace_TFOR_pca_rdy, rna_counts_rdy, 
                                        scoring=scoring, cv=cv_sets, 
                                        return_train_score=True, n_jobs=num_CVs)

# Report CV scores
print('\nCV scores:')
report_score(scores, 'train_explained_variance')
report_score(scores, 'train_r2_score')
report_score(scores, 'train_mean_squared_error')
report_score(scores, 'test_explained_variance')
report_score(scores, 'test_r2_score')
report_score(scores, 'test_mean_squared_error')

In [ ]:
### Regression Plot

# Single prediction
svr.fit(X_train, y_train.ravel())
y_train_pred = svr.predict(X_train)
y_test_pred = svr.predict(X_test)

# Prep plot
fig, ax = plt.subplots(1, 2, figsize=(6,3), sharey=True)

# Create plot
ax[0].scatter(y_train, y_train_pred, color='cyan', edgecolor='darkcyan', alpha=0.5)
ax[1].scatter(y_test, y_test_pred, color='cyan', edgecolor='darkcyan', alpha=0.5)

# Reference line
max_count = rna_counts_rdy.max()
ax[0].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)
ax[1].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)

# Axis adjustments
ax[0].set_xlim([0, max_count])
ax[0].set_ylim([0, max_count])
ax[1].set_xlim([0, max_count])
ax[1].set_ylim([0, max_count])

# Labeling
ax[0].set_title('Training Data (TFOR)')
ax[0].set_xlabel('Ground Truth')
ax[0].set_ylabel('Predicted')
ax[1].set_title('Test Data (TFOR)')
ax[1].set_xlabel('Ground Truth')

# Done
plt.tight_layout()
plt.show()

#### Predicting expression from CFOR

In [ ]:
### Prepare single train-test split for parametrization/visualization

# Split
out = model_selection.train_test_split(fspace_CFOR_pca_rdy, rna_counts_rdy, 
                                       test_size=test_size, random_state=42)
X_train, X_test, y_train, y_test = out

# Report
print "Final source fspace (full, train, test):", fspace_CFOR_pca_rdy.shape, X_train.shape, X_test.shape
print "Final target fspace (full, train, test):", rna_counts_rdy.shape, y_train.shape, y_test.shape

In [ ]:
# Hyperparam screening for SVR

# Param grid
gd = 1.0 / X_test.shape[1]
param_grid = [{'C': [0.01, 0.1, 1.0, 10.0, 100.0], 
               'epsilon': [0.01, 0.1, 0.5, 1.0], 
               'gamma': [gd*10.0, gd, gd*0.1, gd*0.01]}]

# Prep regressor
svr = svm.SVR(kernel='rbf')

# Run grid search
clf = model_selection.GridSearchCV(svr, param_grid, 
                                   cv=cv_sets, scoring=scoring['explained_variance'],
                                   n_jobs=6, verbose=2)
clf.fit(fspace_CFOR_pca_rdy, rna_counts_rdy.ravel())

# Report
print "Best estimator:", clf.best_estimator_
print "Best score:", clf.best_score_

# Use best estimator for cross validation
svr = clf.best_estimator_
scores = model_selection.cross_validate(svr, fspace_CFOR_pca_rdy, rna_counts_rdy, 
                                        scoring=scoring, cv=cv_sets, 
                                        return_train_score=True, n_jobs=num_CVs)

# Report CV scores
print('\nCV scores:')
report_score(scores, 'train_explained_variance')
report_score(scores, 'train_r2_score')
report_score(scores, 'train_mean_squared_error')
report_score(scores, 'test_explained_variance')
report_score(scores, 'test_r2_score')
report_score(scores, 'test_mean_squared_error')

In [ ]:
### Regression Plot

# Single prediction
svr.fit(X_train, y_train.ravel())
y_train_pred = svr.predict(X_train)
y_test_pred = svr.predict(X_test)

# Prep plot
fig, ax = plt.subplots(1, 2, figsize=(6,3), sharey=True)

# Create plot
ax[0].scatter(y_train, y_train_pred, color='cyan', edgecolor='darkcyan', alpha=0.5)
ax[1].scatter(y_test, y_test_pred, color='cyan', edgecolor='darkcyan', alpha=0.5)

# Reference line
max_count = rna_counts_rdy.max()
ax[0].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)
ax[1].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)

# Axis adjustments
ax[0].set_xlim([0, max_count])
ax[0].set_ylim([0, max_count])
ax[1].set_xlim([0, max_count])
ax[1].set_ylim([0, max_count])

# Labeling
ax[0].set_title('Training Data (CFOR)')
ax[0].set_xlabel('Ground Truth')
ax[0].set_ylabel('Predicted')
ax[1].set_title('Test Data (CFOR)')
ax[1].set_xlabel('Ground Truth')

# Done
plt.tight_layout()
plt.show()

#### Predicting expression from position

In [ ]:
### Prepare single train-test split for parametrization/visualization

# Split
out = model_selection.train_test_split(centroids_z_rdy, rna_counts_rdy,
                                       test_size=test_size, random_state=42)
X_train, X_test, y_train, y_test = out

# Report
print "Final source fspace (full, train, test):", centroids_z_rdy.shape, X_train.shape, X_test.shape
print "Final target fspace (full, train, test):", rna_counts_rdy.shape, y_train.shape, y_test.shape

In [ ]:
# Hyperparam screening for SVR

# Param grid
gd = 1.0 / X_test.shape[1]
param_grid = [{'C': [0.01, 0.1, 1.0, 10.0, 100.0], 
               'epsilon': [0.01, 0.1, 0.5, 1.0], 
               'gamma': [gd*10.0, gd, gd*0.1, gd*0.01]}]

# Prep regressor
svr = svm.SVR(kernel='rbf')

# Run grid search
clf = model_selection.GridSearchCV(svr, param_grid, 
                                   cv=cv_sets, scoring=scoring['explained_variance'],
                                   n_jobs=6, verbose=2)
clf.fit(centroids_z_rdy, rna_counts_rdy.ravel())
        
# Report
print "Best estimator:", clf.best_estimator_
print "Best score:", clf.best_score_

# Use best estimator for cross validation
svr = clf.best_estimator_
scores = model_selection.cross_validate(svr, centroids_z_rdy, rna_counts_rdy, 
                                        scoring=scoring, cv=cv_sets, 
                                        return_train_score=True, n_jobs=num_CVs)

# Report CV scores
print('\nCV scores:')
report_score(scores, 'train_explained_variance')
report_score(scores, 'train_r2_score')
report_score(scores, 'train_mean_squared_error')
report_score(scores, 'test_explained_variance')
report_score(scores, 'test_r2_score')
report_score(scores, 'test_mean_squared_error')

In [ ]:
### Regression Plot

# Single prediction
svr.fit(X_train, y_train.ravel())
y_train_pred = svr.predict(X_train)
y_test_pred = svr.predict(X_test)

# Prep plot
fig, ax = plt.subplots(1, 2, figsize=(6,3), sharey=True)

# Create plot
ax[0].scatter(y_train, y_train_pred, color='cyan', edgecolor='darkcyan', alpha=0.5)
ax[1].scatter(y_test, y_test_pred, color='cyan', edgecolor='darkcyan', alpha=0.5)

# Reference line
max_count = rna_counts_rdy.max()
ax[0].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)
ax[1].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)

# Axis adjustments
ax[0].set_xlim([0, max_count])
ax[0].set_ylim([0, max_count])
ax[1].set_xlim([0, max_count])
ax[1].set_ylim([0, max_count])

# Labeling
ax[0].set_title('Training Data (Location)')
ax[0].set_xlabel('Ground Truth')
ax[0].set_ylabel('Predicted')
ax[1].set_title('Test Data (Location)')
ax[1].set_xlabel('Ground Truth')

# Done
plt.tight_layout()
plt.show()

#### Predicting expression from TFOR+CFOR+position

In [ ]:
### Prep combined data data

# Combine
fspace_combined = np.concatenate([fspace_TFOR_pca_rdy, fspace_CFOR_pca_rdy, centroids_z_rdy], axis=1)

In [ ]:
### Prepare single train-test split for parametrization/visualization

# Split
out = model_selection.train_test_split(fspace_combined, rna_counts_rdy,
                                       test_size=test_size, random_state=42)
X_train, X_test, y_train, y_test = out

# Report
print "Final source fspace (full, train, test):", fspace_combined.shape, X_train.shape, X_test.shape
print "Final target fspace (full, train, test):", rna_counts_rdy.shape, y_train.shape, y_test.shape

In [ ]:
# Hyperparam screening for SVR

# Param grid
gd = 1.0 / X_test.shape[1]
param_grid = [{'C': [0.01, 0.1, 1.0, 10.0, 100.0], 
               'epsilon': [0.01, 0.1, 0.5, 1.0], 
               'gamma': [gd*10.0, gd, gd*0.1, gd*0.01]}]

# Prep regressor
svr = svm.SVR(kernel='rbf')

# Run grid search
clf = model_selection.GridSearchCV(svr, param_grid, 
                                   cv=cv_sets, scoring=scoring['explained_variance'],
                                   n_jobs=6, verbose=2)
clf.fit(fspace_combined, rna_counts_rdy.ravel())
        
# Report
print "Best estimator:", clf.best_estimator_
print "Best score:", clf.best_score_

# Use best estimator for cross validation
svr = clf.best_estimator_
scores = model_selection.cross_validate(svr, fspace_combined, rna_counts_rdy, 
                                        scoring=scoring, cv=cv_sets, 
                                        return_train_score=True, n_jobs=num_CVs)

# Report CV scores
print('\nCV scores:')
report_score(scores, 'train_explained_variance')
report_score(scores, 'train_r2_score')
report_score(scores, 'train_mean_squared_error')
report_score(scores, 'test_explained_variance')
report_score(scores, 'test_r2_score')
report_score(scores, 'test_mean_squared_error')

In [ ]:
### Regression Plot

# Single prediction
svr.fit(X_train, y_train.ravel())
y_train_pred = svr.predict(X_train)
y_test_pred = svr.predict(X_test)

# Prep plot
fig, ax = plt.subplots(1, 2, figsize=(6,3), sharey=True)

# Create plot
ax[0].scatter(y_train, y_train_pred, color='cyan', edgecolor='darkcyan', alpha=0.5)
ax[1].scatter(y_test, y_test_pred, color='cyan', edgecolor='darkcyan', alpha=0.5)

# Reference line
max_count = rna_counts_rdy.max()
ax[0].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)
ax[1].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)

# Axis adjustments
ax[0].set_xlim([0, max_count])
ax[0].set_ylim([0, max_count])
ax[1].set_xlim([0, max_count])
ax[1].set_ylim([0, max_count])

# Labeling
ax[0].set_title('Training Data (COMBINED)')
ax[0].set_xlabel('Ground Truth')
ax[0].set_ylabel('Predicted')
ax[1].set_title('Test Data (COMBINED)')
ax[1].set_xlabel('Ground Truth')

# Done
plt.tight_layout()
plt.show()

In [ ]:
# Pretty regression plot for publication

# Single prediction
svr.fit(X_train, y_train.ravel())
y_train_pred = svr.predict(X_train)
y_test_pred = svr.predict(X_test)

# Prep plot
fig, ax = plt.subplots(1, 2, figsize=(6, 3.2), sharey=True)

# Create plot
ax[0].scatter(y_train, y_train_pred, 
              color='midnightblue', edgecolor='', alpha=0.3, s=5)
ax[1].scatter(y_test, y_test_pred, 
              color='midnightblue', edgecolor='', alpha=0.3, s=5)

# Reference line
max_count = rna_counts_rdy.max()
ax[0].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)
ax[1].plot([0,max_count], [0,max_count], '-', c='0.75', zorder=0)

# Crop off and add cropped points back as arrows
crop = 60
if np.any(y_train_pred>crop) or np.any(y_test_pred>crop):
    raise ValueError('Some predicted values are higher than `crop`!')
ax[0].scatter([crop-0.5 for i in range(np.sum(y_train[:,0]>60))], y_train_pred[y_train[:,0]>60],
              color='midnightblue', edgecolor='', alpha=0.5, s=10, marker='>')
ax[1].scatter([crop-0.5 for i in range(np.sum(y_test[:,0]>60))], y_test_pred[y_test[:,0]>60],
              color='midnightblue', edgecolor='', alpha=0.5, s=10, marker='>')

# Axis adjustments
ax[0].set_xlim([0, crop])
ax[0].set_ylim([0, crop])
ax[1].set_xlim([0, crop])
ax[1].set_ylim([0, crop])

# Axis cosmetics
ax[0].yaxis.set_ticks_position('left')
ax[0].xaxis.set_ticks_position('bottom')
ax[1].yaxis.set_ticks_position('left')
ax[1].xaxis.set_ticks_position('bottom')

# Labeling & other cosmetics
ax[0].set_title('Training Data')
ax[0].set_xlabel('$\it{pea3}$ counts (ground truth)')
ax[0].set_ylabel('$\it{pea3}$ counts (predicted)')
ax[1].set_title('Test Data')
ax[1].set_xlabel('$\it{pea3}$ counts (ground truth)')
plt.tight_layout()

# Done
plt.show()

<a id=atlas_run></a>

## 6. Predicting Expression from Shape: Running
[back to top](#top)

----

In [ ]:
### Load and prepare full live-imaged shape space

# Prep loader
expA_loader = ld.DataLoaderIDR()
expA_loader.find_imports(r"data/experimentA/extracted_measurements/", recurse=True, verbose=True)

# Import shape spaces
expA_TFOR_pca, expA_IDs, expA_idx = expA_loader.load_dataset("shape_TFOR_pca_measured.tsv")
expA_CFOR_pca, _, _ = expA_loader.load_dataset("shape_CFOR_pca_measured.tsv", IDs=expA_IDs)
print "Imported TFOR shape space of shape:", expA_TFOR_pca.shape
print "Imported CFOR shape space of shape:", expA_CFOR_pca.shape

# Import TFOR centroid locations
expA_centroids = expA_loader.load_dataset("_other_measurements.tsv", IDs=expA_IDs)[0][:,3:6][:,::-1]
print "Imported TFOR centroids of shape:", expA_centroids.shape
expA_centroids_z = StandardScaler().fit_transform(expA_centroids)

# Combine
expA_combined = np.concatenate([expA_TFOR_pca[:,:use_PCs], expA_CFOR_pca[:,:use_PCs], expA_centroids_z], axis=1)

# Report
print expA_TFOR_pca.shape, expA_CFOR_pca.shape, expA_centroids_z.shape, expA_combined.shape

In [ ]:
### Run best possible smFISH count prediction for entire atlas

# Prepare the best regressor 
svr = svm.SVR(kernel='rbf', C=10.0, epsilon=0.01, gamma = 1.0 / X_test.shape[1] * 0.1)

# Train based on entire smFISH dataset
svr.fit(fspace_combined, rna_counts_rdy.ravel())

# Predict for entire atlas
expA_counts = svr.predict(expA_combined)

# Set the occasional negative count to zero
expA_counts[expA_counts < 0.0] = 0.0

<a id=atlas_viz></a>

## 7. Predicting Expression from Shape: Visualization
[back to top](#top)

----

In [ ]:
### QC: Compare predicted atlas counts to measured counts

# Note:
#  This looks quite good. The prediction obviously doesn't capture the long 
#  tail of the real measurements, which also pulls the overall average down
#  a bit. This was to be expected and may not even be wrong.

# Get count means
count_means = np.array([np.mean(rna_counts[fspace_idx==prim_idx]) 
                        for prim_idx in range(len(prim_IDs))])
expA_means = np.array([np.mean(expA_counts[expA_idx==prim_idx]) 
                        for prim_idx in range(len(expA_IDs))])

# Fig prep
fig, ax = plt.subplots(1, 2, figsize=(6, 4.5), sharey=True)

# Make boxplots
bp_m = ax[0].boxplot([count_means, expA_means], widths=0.65, patch_artist=True, showfliers=False)
bp_a = ax[1].boxplot([rna_counts, expA_counts], widths=0.65, patch_artist=True, showfliers=False)

# Boxplot styling function (making it similar to Sevi's paper)
def style_boxplot(bp):
    for patch in bp['boxes']:
        patch.set(edgecolor='black', linewidth=1.2,)
    for whisker in bp['whiskers']:
        whisker.set(color='black', linestyle='-')
    for cap in bp['caps']:
        cap.set(linewidth=1.2)
    for median in bp['medians']:
        median.set(color='black', linewidth=1.2)
        
# Style the boxplots
style_boxplot(bp_m)
style_boxplot(bp_a)

# Add scatter
ax[0].scatter(np.random.normal(1.0, 0.06, len(count_means)), count_means, 
              zorder=10, s=20, alpha=0.7, c='midnightblue', edgecolor='')
ax[0].scatter(np.random.normal(2.0, 0.08, len(expA_means)), expA_means, 
              zorder=10, s=20, alpha=0.3, c='purple', edgecolor='')
ax[1].scatter(np.random.normal(1.0, 0.06, len(rna_counts)), rna_counts, 
              zorder=10, s=2, alpha=0.2, c='midnightblue', edgecolor='')
ax[1].scatter(np.random.normal(2.0, 0.10, len(expA_counts)), expA_counts, 
              zorder=10, s=2, alpha=0.05, c='purple', edgecolor='')

# Add arrows for outliers
crop = 50
ax[1].scatter(np.random.normal(1.0, 0.06, np.sum(rna_counts>crop)), 
              [crop-0.5 for i in range(np.sum(rna_counts>crop))],
              color='midnightblue', edgecolor='', alpha=0.2, s=10, marker='^')
if np.any(expA_counts > crop): raise ValueError()

# Set axis limits
ax[0].set_ylim([-2, crop])

# Remove axis ticks
ax[0].yaxis.set_ticks_position('left')
ax[0].xaxis.set_ticks_position('bottom')
ax[1].yaxis.set_ticks_position('left')
ax[1].xaxis.set_ticks_position('bottom')

# Axis labels
from matplotlib import rcParams
rcParams['mathtext.default'] = 'regular'
ax[0].set_ylabel(r'$\it{pea3}$ transcripts per cell', fontsize=16, labelpad=5)
ax[0].set_title('sample means', fontsize=16)
ax[1].set_title('all cells', fontsize=16)
ax[0].set_xticklabels(['smFISH', 'atlas'], fontsize=14)
ax[1].set_xticklabels(['smFISH', 'atlas'], fontsize=14)
ax[0].tick_params(axis='y', which='major', labelsize=14)
plt.tight_layout()

# Print stats
print 'pMWU(means):', stats.mannwhitneyu(count_means, expA_means, alternative='two-sided')[1]
print 'pMWU(all):', stats.mannwhitneyu(rna_counts, expA_counts, alternative='two-sided')[1]

# Show
plt.show()

In [ ]:
### Atlas tissue consensus map

# Settings
xlim = (-130,  8)
ylim = ( -19, 19)

# Get plot values & remove outliers
plot_values = expA_counts

# Tools for smoothing on scatter
from katachi.utilities.pcl_helpers import pcl_gaussian_smooth
from scipy.spatial.distance import pdist, squareform

# Cut off at prim contour outline
kernel_prim = stats.gaussian_kde(expA_centroids[:,1:].T)
f_prim = kernel_prim(expA_centroids[:,1:].T)
f_prim_mask = f_prim > f_prim.min() + (f_prim.max()-f_prim.min())*0.1
plot_values = plot_values[f_prim_mask]
plot_centroids = expA_centroids[f_prim_mask]

# Smoothen?
pdists = squareform(pdist(plot_centroids[:,1:]))
plot_values = pcl_gaussian_smooth(pdists, plot_values[:,np.newaxis], sg_percentile=0.5)[:,0]

# Initialize figure
fig, ax = plt.subplots(1, figsize=(8, 2.8))

# Contourf plot
cfset = ax.tricontourf(plot_centroids[:,2], plot_centroids[:,1], plot_values, 20, 
                       cmap='plasma', vmax=20)  # NOTE: vmax set to be consistent with measured plot!

# Illustrative centroids from a single prim
plt.scatter(expA_centroids[expA_idx==expA_IDs.index(expA_IDs[0]), 2], 
            expA_centroids[expA_idx==expA_IDs.index(expA_IDs[0]), 1],
            c='', alpha=0.5)

# Cosmetics
ax.set_xlabel('TFOR x', fontsize=16)
ax.set_ylabel('TFOR y', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=13)
plt.xlim(xlim); plt.ylim(ylim)
ax.invert_yaxis()  # To match images

# Colorbar
cbar = plt.colorbar(cfset, ax=ax, pad=0.01)
cbar.set_label('RNA Counts', rotation=270, labelpad=15, fontsize=16)
cbar.ax.tick_params(labelsize=13)

# Finalize
plt.tight_layout()
plt.show()

----

[back to top](#top)